[original svg](https://commons.wikimedia.org/wiki/File:Comparison_gender_life_expectancy_CIA_factbook.svg)<br>
[List of countries by population](https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations))

In [1]:
import pandas as pd

In [2]:
# load data for countries
df = pd.read_csv('data/CIA_data_2019.csv', sep='\t', dtype={'label':'Int64'}, index_col=1)

# remove records about countries where there is no records about male and female life expectancy
df.dropna(subset=['male', 'female'], how='any', inplace=True)
df = df[['population', 'all', 'male', 'female', 'region', 'label']]
df.index.name = ''

df

,population,all,male,female,region,label
,,,,,,
WORLD,7405107650,69.0,67.0,71.1,-,-1
EUROPEAN UNION,517111329,80.7,77.9,83.6,-,-20
China,1384688986,75.8,73.7,78.1,ea,20
India,1296834042,69.1,67.8,70.5,as,25
USA,329256465,80.1,77.8,82.3,na,28
...,...,...,...,...,...,...
Cook Islands,9038,76.2,73.4,79.2,oc,<NA>
"Saint Helena, Ascension, and Tristan da Cunha",7841,79.8,76.8,82.9,af,<NA>
Saint Pierre and Miquelon,5471,80.7,78.4,83.2,na,<NA>


In [3]:
# exclude countries with population less than 1_000_000
df = df[df.population >= 1_000_000]
df

,population,all,male,female,region,label
,,,,,,
WORLD,7405107650,69.0,67.0,71.1,-,-1
EUROPEAN UNION,517111329,80.7,77.9,83.6,-,-20
China,1384688986,75.8,73.7,78.1,ea,20
India,1296834042,69.1,67.8,70.5,as,25
USA,329256465,80.1,77.8,82.3,na,28
...,...,...,...,...,...,...
Timor-Leste,1321929,68.7,67.1,70.4,ea,<NA>
Estonia,1244288,77.0,72.3,82.0,eu,<NA>
Cyprus,1237088,79.0,76.2,81.9,eu,<NA>


In [4]:
# load instructins how data should be handled
handling = pd.read_csv('data/CIA_handling.csv', index_col=0)

# transform to dictionary
colors, regions = handling.to_dict('dict').values()
del handling
colors

{'na': '#0033cc',
 'la': '#0099cc',
 'sa': '#009900',
 'eu': '#6600cc',
 'af': '#000000',
 'me': '#cc9900',
 'ca': '#cc00cc',
 'as': '#993300',
 'ea': '#ff0000',
 'oc': '#0000ff',
 '-': '#999999'}

In [5]:
regions

{'na': 'North America',
 'la': 'Central America',
 'sa': 'South America',
 'eu': 'Europe',
 'af': 'Africa',
 'me': 'Middle East',
 'ca': 'Central Asia',
 'as': 'South Asia',
 'ea': 'East/SE Asia',
 'oc': 'Australia/Oceania',
 '-': 'Other'}

In [6]:
outs = {k: '' for k in regions.keys()}
outs

{'na': '',
 'la': '',
 'sa': '',
 'eu': '',
 'af': '',
 'me': '',
 'ca': '',
 'as': '',
 'ea': '',
 'oc': '',
 '-': ''}

<br />
<br />

In [7]:
out_defs = out_graph = ''

for i in range(len(df)):
    # extract single values for country
    population, le_a, le_m, le_f, region, offset_label = df.iloc[i]
    name = df.iloc[i].name
    x = int(le_m * 10)
    y = int(le_f * -10)
    radius = round(population ** (1/3) / 20, 2)
    color = colors[region]
    
    if pd.notna(offset_label):  # if there is label for country
        y_text = y + offset_label
        x_text = x + (1 if offset_label > 0 else -1) * \
                 (2 + int(((radius ** 2 - offset_label ** 2) ** 0.5 if radius > abs(offset_label) else 0) + 0.5));
        text_anchor = 'start' if offset_label > 0 else 'end';
        out_label_defs = f'\n   <text x="{x_text}" y="{y_text}" dy="0.7ex" text-anchor="{text_anchor}" stroke="none">{name}</text>'
        out_label_active = "";
    else:
        x_text = int(x - radius * 0.8);
        y_text = int(y - radius * 0.8);
        out_label_defs = "";
        out_label_active = f'\n   <text x="{x_text}" y="{y_text}" dy="0.7ex" class="hidden_text" stroke="{color}">{name}</text>'
    
    population = f"{population:,}".replace(',', ' ')
    
    out_defs += (
        f'\n  <g id="bubble_{i}" fill="{color}">{out_label_defs}'
        f'\n   <circle cx="{x}" cy="{y}" r="1"/>'
        f'\n   <circle cx="{x}" cy="{y}" r="{radius}" stroke="{color}" fill="url(#grad_{region})"/>'
        f'\n  </g>')
    
    out_graph += (
        f'\n  <g class="active">'
        f'\n   <use xlink:href="#bubble_{i}"/>{out_label_active}'
        f'\n   <title>{name} (population: {population})\nfemale: {le_f}, male: {le_m}, overall: {le_a} years</title>'
        f'\n  </g>')
    
    outs[region] += f'\n    <use xlink:href="#bubble_{i}"/>'

In [8]:
out_grad = out_legend = ''
y = -935

for abbr in regions.keys():
    # print(abbr)
    color = colors[abbr]
    region = regions[abbr]

    out_grad += (
        f'\n  <radialGradient id="grad_{abbr}" cx="50%" cy="50%" r="50%" fx="25%" fy="25%">'
        f'\n   <stop offset="50%" stop-color="{color}" stop-opacity="0"/>'
        f'\n   <stop offset="70%" stop-color="{color}" stop-opacity="0.05"/>'
        f'\n   <stop offset="99%" stop-color="{color}" stop-opacity="0.25"/>'
        f'\n  </radialGradient>')
    
    x = y + 5
    out_legend += (
        f'\n  <g class="active">'
        f'\n    <text x="425" y="{x}" fill="{color}">{region}</text>'
        f'\n    <circle cx="415" cy="{y}" r="8" fill="url(#grad_{abbr})"/>'
        f'\n    <circle cx="415" cy="{y}" r="1" fill="{color}"/>'
        + outs[abbr] +
        f'\n  </g>')

    y += 19

In [9]:
out_ticks = ''

for tick in range(45, 91, 5):
    x = tick * 10
    out_ticks += f'\n    <text x="{x}" y="-483">{tick}</text>'

for tick in range(50, 96, 5):
    y = tick * -10 + 10
    out_ticks += f'\n    <text x="385" y="{y}">{tick}</text>'
    
print(out_ticks)


    <text x="450" y="-483">45</text>
    <text x="500" y="-483">50</text>
    <text x="550" y="-483">55</text>
    <text x="600" y="-483">60</text>
    <text x="650" y="-483">65</text>
    <text x="700" y="-483">70</text>
    <text x="750" y="-483">75</text>
    <text x="800" y="-483">80</text>
    <text x="850" y="-483">85</text>
    <text x="900" y="-483">90</text>
    <text x="385" y="-490">50</text>
    <text x="385" y="-540">55</text>
    <text x="385" y="-590">60</text>
    <text x="385" y="-640">65</text>
    <text x="385" y="-690">70</text>
    <text x="385" y="-740">75</text>
    <text x="385" y="-790">80</text>
    <text x="385" y="-840">85</text>
    <text x="385" y="-890">90</text>
    <text x="385" y="-940">95</text>


In [10]:
svg_code = """<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="100%" height="100%" viewBox="350 -953 560 490">
 <title>CIA-2019</title>
 <desc>My description</desc>
 <style type="text/css">
  #main         { font-family:Helvetica,Arial,sans-serif; font-size:16px; text-anchor:middle;
                  stroke-opacity:0; fill-opacity:0.5; cursor:default; }
  #main:hover   { stroke-opacity:0; fill-opacity:0.1; }
  .nofade,
  .active:hover { stroke-opacity:1; fill-opacity:1; }
  .hidden_text  { font-size:14px; text-anchor:end; fill:none; pointer-events:none; }
 </style>
 <defs>
 
  <!-- determine appearance of circles -->""" + \
    out_grad + \
"""

  <!-- determine each country -->""" + \
    out_defs + \
"""

  <!-- vertical grid -->
  <path id="grid_x" d="M 0 -500 V -950"/>
  <g id="grid_x_5">
   <use xlink:href="#grid_x"/>
   <use xlink:href="#grid_x" transform="translate(10,0)"/>
   <use xlink:href="#grid_x" transform="translate(20,0)"/>
   <use xlink:href="#grid_x" transform="translate(30,0)"/>
   <use xlink:href="#grid_x" transform="translate(40,0)"/>
   <use xlink:href="#grid_x" transform="translate(50,0)"/>
  </g>
  
  <!-- horizontal grid -->
  <path id="grid_y" d="M 400 0 H 900"/>
  <g id="grid_y_5">
   <use xlink:href="#grid_y"/>
   <use xlink:href="#grid_y" transform="translate(0,-10)"/>
   <use xlink:href="#grid_y" transform="translate(0,-20)"/>
   <use xlink:href="#grid_y" transform="translate(0,-30)"/>
   <use xlink:href="#grid_y" transform="translate(0,-40)"/>
   <use xlink:href="#grid_y" transform="translate(0,-50)"/>
  </g>
 </defs>



 <g id="main">
  <g class="nofade">
   <!-- background for the whole diagram -->
   <circle cx="0" cy="0" r="99999" fill="#ffffff"/>

   <!-- draw grid -->
   <g stroke-opacity="0.05" stroke="#000000">
    <use xlink:href="#grid_x_5" transform="translate(400,0)"/>
    <use xlink:href="#grid_x_5" transform="translate(450,0)"/>
    <use xlink:href="#grid_x_5" transform="translate(500,0)"/>
    <use xlink:href="#grid_x_5" transform="translate(550,0)"/>
    <use xlink:href="#grid_x_5" transform="translate(600,0)"/>
    <use xlink:href="#grid_x_5" transform="translate(650,0)"/>
    <use xlink:href="#grid_x_5" transform="translate(700,0)"/>
    <use xlink:href="#grid_x_5" transform="translate(750,0)"/>
    <use xlink:href="#grid_x_5" transform="translate(800,0)"/>
    <use xlink:href="#grid_x_5" transform="translate(850,0)"/>
    <use xlink:href="#grid_y_5" transform="translate(0,-500)"/>
    <use xlink:href="#grid_y_5" transform="translate(0,-550)"/>
    <use xlink:href="#grid_y_5" transform="translate(0,-600)"/>
    <use xlink:href="#grid_y_5" transform="translate(0,-650)"/>
    <use xlink:href="#grid_y_5" transform="translate(0,-700)"/>
    <use xlink:href="#grid_y_5" transform="translate(0,-750)"/>
    <use xlink:href="#grid_y_5" transform="translate(0,-800)"/>
    <use xlink:href="#grid_y_5" transform="translate(0,-850)"/>
    <use xlink:href="#grid_y_5" transform="translate(0,-900)"/>
   </g>
   
   <!-- labels for axes -->
   <g fill="#000000">  <!-- color of font -->
    <text x="650" y="-467">Male life expectancy at birth in years</text>
    <text transform="rotate(-90)" x="750" y="365">Female life expectancy at birth in years</text>""" + \
    out_ticks + \
"""
   </g>
   
   <!-- diagonal line -->
   <path d="M 500 -500 L 900 -900" stroke="#00cc00" stroke-width="2" stroke-dasharray="2,1"/>
  </g>

  <!-- menu -->
  <g text-anchor="start">
  
   <!-- rectangle around items -->
   <rect class="nofade" x="403" y="-945" width="154" height="210" rx="5" ry="5" stroke="#999999" fill="#ffffff"/>
   
   <!-- single items -->""" + \
    out_legend + \
"""
  </g>

  <!-- pop-up notes -->""" + \
    out_graph + \
"""
 </g>
</svg>"""

with open('output\CIA -filtered.svg', 'w') as fh:
    fh.write(svg_code)

print('done')

done
